In [4]:
import pickle
import random
import time
import os

import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
from matplotlib import pyplot as plt
import soundfile as sf
import librosa
import librosa.display

tqdm.pandas()

import paule
from paule.util import (speak, normalize_cp, inv_normalize_cp, normalize_mel_librosa, inv_normalize_mel_librosa, stereo_to_mono, librosa_melspec, pad_same_to_even_seq_length, RMSELoss, mel_to_sig, pad_batch_online)
from models import *
from matplotlib import cm

DIR = os.getcwd()
DEVICE = torch.device('cpu')

In [5]:
DATA_DIR = "/Volumes/T7/Common_Voice/data/"
RESULT_DICT = "/Volumes/T7/Common_Voice/common_voice_test_planning"
SAVE_DICT = "/Volumes/T7/Common_Voice/common_voice_test_planning/results_acoustic_lr_001_lr_model_00001_10_outer_50_inner_6_batches_2_epochs_only_new"

# Initialize Paule

In [7]:
paule_model = paule.Paule(device=DEVICE)

# Prepare Test Data

In [ ]:
test_data = pd.read_pickle(DATA_DIR + "common_voice_geco_words_test_subset_slim_prot4.pkl")

## Create Flac Files 

In [ ]:
"""
for i,row in test_data.iterrows():
    sr = 48000
    sig = row.wav_rec
    file_name = row.file_name
    label = row.label
    
    path = RESULT_DICT + label + "/"
    if os.path.isdir(path):
        file = path +  file_name + "_" + label + ".flac"
        if file.is_file():
            j = 1
            while file.is_file(): 
                file = path +  file_name + "_" + label + "%d.flac" % j
                j+=1
        sf.write(path +  file_name + "_" + label + "3.flac" ,sig, sr)
    else:
        os.mkdir(path)
        sf.write(path +  file_name + "_" + label + ".flac" ,sig, sr)
"""

## Random Shuffle Files

In [ ]:
files = []
for dic in os.listdir(RESULT_DICT):
    if (".DS_Store" in dic) or ("results" in dic) : # .DS_Store stores custom attributes of its containing folder in macOS
        continue
    else:
        if not os.path.isdir(os.path.join(SAVE_DICT, dic)):
            os.mkdir(os.path.join(SAVE_DICT, dic))
        path = os.path.join(RESULT_DICT, dic)
        for file in os.listdir(path):
            if (".DS_Store" in file) or ("._" in file):
                continue
            else:
                file = os.path.join(path,file)
                files.append(file)
random.seed(30112021)
random.shuffle(files)

### Omit already planned files

In [ ]:
already_planned_files = []
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if "planned" in file: 
                    planned_file = "_".join(file.split("_")[:-1]) + ".flac" 
                    already_planned_files.append(planned_file)

unplanned_files = [file for file in files if file.split("/")[-1] not in already_planned_files]

### Load predictive model already used and further learned during planning 

In [ ]:
# load model
pred_model = torch.load(f"{save_file}_pred_model.pt", map_location=DEVICE)
optimizer = torch.load(f"{save_file}_pred_optimizer.pt", map_location=DEVICE)

In [ ]:
paule_model.pred_model = model
paule_model.pred_optimizer = optimizer

# Start Planning and store results 

In [ ]:
for i, file in enumerate(unplanned_files):
    i += 0
    target_acoustic = file #path + "/" + file
    save_file = f"{SAVE_DICT}/{os.path.dirname(file).split('/')[-1]}/{os.path.basename(file)[:-5]}"
    
    results = paule_model.plan_resynth(learning_rate_planning=0.01,
            learning_rate_learning=0.001,
            target_acoustic=target_acoustic,
            initialize_from="acoustic",
            objective="acoustic",
            n_outer=20, n_inner=50,
            continue_learning=True,
            add_training_data=False,
            log_ii=1,
            log_semantics=True,
            n_batches=6, batch_size=8, n_epochs=5,
            log_gradients=False,
            plot=True, plot_save_file=save_file, seed=None,
            verbose=True)
    
    # save model and optimizer
    torch.save(paule_model.pred_model, f"{save_file}_pred_model.pt")
    torch.save(paule_model.pred_optimizer, f"{save_file}_pred_optimizer.pt")

    # save results without model and optimizer
    with open(f"{save_file}.pkl", 'wb') as pfile:
        pickle.dump(results, pfile)


    # save initial and planned flac
    prod_sr = results.prod_sr
    sig_initial = results.sig_steps[0]
    sf.write(save_file + "_initial.flac", sig_initial, prod_sr)
    prod_sig = results.prod_sig
    sf.write(save_file + "_planned.flac", prod_sig, prod_sr)

    # save svgs
    planned_cp = results.planned_cp
    path = save_file + '_svgs/'
    if not os.path.exists(path):
        os.mkdir(path)
    util.export_svgs(util.inv_normalize_cp(planned_cp), path=path)

    # ffmpeg -r 80 -width 600 -i tract%05d.svg -i planned_0.flac planned_0.mp4
    # /usr/bin/ffmpeg -r 80 -width 600 -i /home/tino/Documents/phd/projects/paule/results/000003-Wissenschaft_svgs/tract%05d.svg -i results/000003-Wissenschaft_planned.flac planned.mp4


    # save loss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.planned_loss_steps, label="planned loss", c="C0")
    ax.legend()
    fig.savefig(f"{save_file}_loss.png")

    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.prod_loss_steps, label="produced mel loss", c="C1")
    ax.plot(results.planned_mel_loss_steps, label="planned mel loss", c="C0")
    ax.legend()
    fig.savefig(f"{save_file}_loss_mel.png")

    # save subloss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.vel_loss_steps, label="vel loss", c="C2")
    ax.plot(results.jerk_loss_steps, label="jerk loss", c="C3")
    ax.legend()
    fig.savefig(f"{save_file}_loss_subloss.png")

    # save semvec loss plot
    fig, ax = plt.subplots(figsize=(15, 8), facecolor="white")
    ax.plot(results.pred_semvec_loss_steps, label="planned semvec loss", c="C0")
    ax.plot(results.prod_semvec_loss_steps, label="produced semvec loss", c="C1")
    ax.legend()
    fig.savefig(f"{save_file}_loss_semvec.png")


# Plot Model Loss for continued learning 

In [ ]:
model_loss = []
for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file: 
                    with open(os.path.join(path,file), 'rb') as f:
                        results = pickle.load(f)
                        model_loss += list((np.asarray(results[21][:-1:2]) + np.asarray(results[21][1::2]))/2)#results[21]

In [ ]:
fig, ax = plt.subplots(figsize = (15,8), facecolor = "white")
ax.plot(model_loss, label = "model loss", c = "C0")
#ax.plot(range(len(model_loss)),np.repeat(np.mean(model_loss),len(model_loss)) ,c = "C1")
ax.legend()

# Data Wrangling with Final Results

In [ ]:
final_results = pd.DataFrame(columns=['file_name','label', 'planned_cp', 'inv_cp', 'target_sig','inv_sig', 'prod_sig', 'target_mel', 
                                      'inv_mel','prod_mel','pred_mel', 'prod_mel_loss', 'pred_mel_loss', 'vel_loss', 'jerk_loss',
                                     'prod_semvec','pred_semvec','prod_semvec_loss', 'pred_semvec_loss'])
ix = 0

for dic in os.listdir(SAVE_DICT):
    if ".DS_Store" in dic:
        continue
    else:
        path = os.path.join(SAVE_DICT,dic)
        if os.path.isdir(path):
            for file in os.listdir(path):
                if ".pkl" in file: 
                    with open(os.path.join(path,file), 'rb') as f:
                        results = pickle.load(f)
                        
                        file_name = "_".join(file.split("_")[:-1])
                        label=file.split("_")[-1][:-4]
                        planned_cp = results[0]
                        inv_cp = results[1]
                        target_sig = results[2]
                        prod_sig = results[4]
                        target_mel = results[3]
                        
                        inv_sig, sr = speak(inv_normalize_cp(inv_cp))
                        inv_mel = librosa_melspec(inv_sig, sr)
                        inv_mel = normalize_mel_librosa(inv_mel)
                        prod_mel = results[5]
                        pred_mel = results[6]
                        prod_mel_loss = results[7][-1]
                        pred_mel_loss = results[9][-1]
                        vel_loss = results[10][-1]
                        jerk_loss = results[11][-1]
                        prod_semvec = results[16][-1]
                        pred_semvec = results[15][-1]
                        prod_semvec_loss = results[13][-1]
                        pred_semvec_loss = results[12][-1]
                        
                        
                        final_results.loc[ix] = [file_name,label,planned_cp,inv_cp,target_sig,inv_sig,prod_sig,target_mel,inv_mel,prod_mel,
                                                pred_mel,prod_mel_loss,pred_mel_loss,vel_loss,jerk_loss,prod_semvec,
                                                pred_semvec,prod_semvec_loss,pred_semvec_loss]
                        ix+=1

In [ ]:
#final_results.to_pickle(SAVE_DICT+"/final_results.pkl", protocol=4)